In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objs as go
import io

: 

: 

In [ ]:
uploaded = files.upload()


Saving CLEAN_2010_2022.csv to CLEAN_2010_2022.csv


In [ ]:
uploaded_file = io.BytesIO(uploaded["CLEAN_2010_2022.csv"])

df = pd.read_csv(uploaded_file)

df.head()

,Date,SMR CV,SMR L,SMR 5,SMR GP,SMR 10,SMR 20,Bulk Latex
0,2010-01-04,1004.5,986.0,976.0,973.0,956.5,954.5,606.0
1,2010-01-05,1015.5,995.5,988.0,985.0,968.5,966.5,613.5
2,2010-01-06,1022.0,1005.5,995.0,992.0,975.0,973.0,620.5
3,2010-01-07,1026.5,1013.5,1003.5,1000.5,983.5,981.5,627.5
4,2010-01-08,1032.5,1015.5,1004.0,1001.0,984.0,982.0,631.0


In [ ]:
df = df[["Date","Bulk Latex"]]
df.tail()

,Date,Bulk Latex
3156,2022-12-19,472.5
3157,2022-12-20,477.0
3158,2022-12-21,480.0
3159,2022-12-22,481.5
3160,2022-12-23,482.0


In [ ]:
close_data = df['Bulk Latex'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

2528
633


In [ ]:
np.random.seed(0)  
tf.random.set_seed(0)

In [ ]:
look_back = 2

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

model = Sequential()
model.add(
    LSTM(100,
        activation='relu',
        input_shape=(look_back,1), seed=42)
)

model.add(Dense(25))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 10

while (True):
  model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=num_epochs,
      verbose=0
  )
  prediction = model.predict(test_generator, verbose=0)
  close_test_check = close_test[:-2] 

  mse = mean_squared_error(close_test_check, prediction)
  # Calculate the root mean squared error
  rmse = np.sqrt(mse)

  print(f'RMSE: {rmse}')
  if rmse <= 4:
    break


RMSE: 3.474200289472257


In [ ]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates



In [ ]:
num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 20ms/step


In [ ]:
forecast_dates

In [ ]:
forecast

array([482.        , 481.65753174, 481.73855591, 481.60644531,
       481.58132935, 481.50216675, 481.45007324, 481.38421631,
       481.32507324, 481.26242065, 481.20132446, 481.13928223,
       481.07757568, 481.01547241, 480.95346069, 480.8911438 ,
       480.8288269 , 480.7663269 , 480.70373535, 480.64102173,
       480.57821655, 480.51522827, 480.45220947, 480.38897705,
       480.32565308, 480.26220703, 480.19863892, 480.13494873,
       480.07116699, 480.00723267, 479.94314575])

In [ ]:
forecast

array([482.        , 469.95620728, 470.49307251, 470.88964844,
       469.6824646 , 469.59698486, 466.58929443, 465.78533936,
       465.02200317, 463.66430664, 462.68045044])

In [ ]:
forecast

array([823.5       , 824.52862549, 825.55786133, 826.58734131,
       827.6171875 , 828.6473999 , 829.67803955, 830.70898438,
       831.74029541, 832.77185059, 833.80389404, 834.83624268,
       835.86895752, 836.9019165 , 837.9352417 , 838.96905518,
       840.00311279, 841.03753662, 842.07226562, 843.10736084,
       844.14276123, 845.1786499 , 846.21472168, 847.25109863,
       848.28796387, 849.32507324, 850.36248779, 851.40020752,
       852.43829346, 853.47668457, 854.51538086])

In [ ]:
forecast

array([823.5       , 804.04290771, 804.51953125, 804.71221924,
       804.17724609, 802.7557373 , 800.48614502, 797.50366211,
       794.00823975, 792.69683838, 791.03881836, 789.144104  ,
       787.03924561, 784.79760742, 782.59552002, 780.5045166 ,
       778.5657959 , 776.56848145, 774.52679443, 772.46160889,
       770.40124512, 768.36810303, 766.35614014, 764.35357666,
       762.34527588, 760.33520508, 758.32940674, 756.33166504,
       754.34277344, 752.35974121, 750.38110352])